In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import seaborn as sns
import networkx as nx
import matplotlib.colors as mcolors
from scipy import stats
from scipy.stats import f_oneway, kruskal
# pd.options.display.max_rows = None
# pd.options.display.max_columns = None
import glob as glob
import math
pd.set_option('display.max_rows', None)  # To display all rows
pd.set_option('display.max_columns', None)  # To display all columns
from scipy.stats import shapiro

In [19]:
#load in master dataset csv 
gm_3q = pd.read_csv('graph_metrics_adni_a4_bootstrapped_3quant.csv')
print(gm_3q['Centiloid Quantile'].unique())

[0 1 2]


In [20]:
adni_gm = gm_3q[gm_3q['Dataset']=='ADNI']
a4_gm = gm_3q[gm_3q['Dataset']=='A4']
print(adni_gm['Dataset'].nunique()) #looks good
print(a4_gm['Dataset'].nunique())

1
1


In [21]:
print(adni_gm.head())

  Dataset  Centiloid Quantile  Weighted Clustering Coefficient  \
0    ADNI                   0                         0.068934   
1    ADNI                   0                         0.083235   
2    ADNI                   0                         0.058079   
3    ADNI                   0                         0.070969   
4    ADNI                   0                         0.067504   

   Weighted Avg. Shortest Path Length  Weighted Small World  
0                            1.842079              0.325390  
1                            2.203289              0.329465  
2                            2.307050              0.207186  
3                            2.092218              0.308219  
4                            2.123366              0.281602  


Do One Way ANOVA test between the quartile groups for weighted clustering coefficent and shortest path leng, do KS for small world 

In [24]:
#run sig test 
#anova for clus coeff and spl 
#kruskil wallis for small world 
#default alpha sig is .05

def perform_anova_kruskal(df1, df2, metrics, group_column):
    results = []
    
    for df, name in [(df1, 'ADNI'), (df2, 'A4')]:
        for metric in metrics:
            group_data = [df[df[group_column] == group][metric] for group in df[group_column].unique()]
            if metric == 'Weighted Small World':
                result = kruskal(*group_data)
            else:
                result = f_oneway(*group_data)
            significant = 'Yes' if result.pvalue < 0.05 else 'No'
            results.append({
                'Dataset': name,
                'Metric': metric,
                'Test': 'Kruskal-Wallis' if metric == 'Weighted Small World' else 'ANOVA',
                'Statistic': result.statistic,
                'p-value': result.pvalue,
                'Significant': significant
            })
                
    return pd.DataFrame(results)

#test
metrics = ['Weighted Clustering Coefficient', 'Weighted Avg. Shortest Path Length', 'Weighted Small World']
results = perform_anova_kruskal(adni_gm, a4_gm, metrics, 'Centiloid Quantile')
results = results.sort_values(by='Metric')
print(results)


  Dataset                              Metric            Test    Statistic  \
1    ADNI  Weighted Avg. Shortest Path Length           ANOVA   598.080717   
4      A4  Weighted Avg. Shortest Path Length           ANOVA  2085.018052   
0    ADNI     Weighted Clustering Coefficient           ANOVA   567.441841   
3      A4     Weighted Clustering Coefficient           ANOVA   570.450427   
2    ADNI                Weighted Small World  Kruskal-Wallis  1528.089792   
5      A4                Weighted Small World  Kruskal-Wallis  1954.486242   

         p-value Significant  
1  2.732263e-219         Yes  
4   0.000000e+00         Yes  
0  1.040341e-209         Yes  
3  1.175259e-210         Yes  
2   0.000000e+00         Yes  
5   0.000000e+00         Yes  


export it in latex table format 

In [28]:
# Convert DataFrame to LaTeX
latex_table = results.to_latex(index=False, escape=False)

# Complete LaTeX code for the table
latex_complete = """
\\begin{table}[htbp]
\\centering
\\caption{Your table caption goes here}
\\label{tab:your_table_label}
\\begin{tabular}{llllll}
\\toprule
%s
\\bottomrule
\\end{tabular}
\\end{table}
""" % latex_table

# Print complete LaTeX code
print(latex_complete)


\begin{table}[htbp]
\centering
\caption{Your table caption goes here}
\label{tab:your_table_label}
\begin{tabular}{llllll}
\toprule
\begin{tabular}{lllrrl}
\toprule
Dataset & Metric & Test & Statistic & p-value & Significant \\
\midrule
ADNI & Weighted Avg. Shortest Path Length & ANOVA & 598.080717 & 0.000000 & Yes \\
A4 & Weighted Avg. Shortest Path Length & ANOVA & 2085.018052 & 0.000000 & Yes \\
ADNI & Weighted Clustering Coefficient & ANOVA & 567.441841 & 0.000000 & Yes \\
A4 & Weighted Clustering Coefficient & ANOVA & 570.450427 & 0.000000 & Yes \\
ADNI & Weighted Small World & Kruskal-Wallis & 1528.089792 & 0.000000 & Yes \\
A4 & Weighted Small World & Kruskal-Wallis & 1954.486242 & 0.000000 & Yes \\
\bottomrule
\end{tabular}

\bottomrule
\end{tabular}
\end{table}



In [29]:
import pandas as pd
import textwrap

# # Example DataFrame
# data = {
#     'Dataset': ['ADNI', 'A4', 'ADNI', 'A4', 'ADNI', 'A4'],
#     'Metric': ['Weighted Avg. Shortest Path Length', 'Weighted Avg. Shortest Path Length',
#                'Weighted Clustering Coefficient', 'Weighted Clustering Coefficient',
#                'Weighted Small World', 'Weighted Small World'],
#     'Test': ['ANOVA', 'ANOVA', 'ANOVA', 'ANOVA', 'Kruskal-Wallis', 'Kruskal-Wallis'],
#     'Statistic': [598.080717, 2085.018052, 567.441841, 570.450427, 1528.089792, 1954.486242],
#     'p-value': [0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
#     'Significant': ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']
# }

# Convert DataFrame to LaTeX
latex_table = results.to_latex(index=False, escape=False)

# Complete LaTeX code for the table
latex_complete = """
\\begin{table}[htbp]
\\centering
\\caption{Your table caption goes here}
\\label{tab:your_table_label}
\\begin{tabular}{llllll}
\\toprule
%s
\\bottomrule
\\end{tabular}
\\end{table}
""" % textwrap.indent(latex_table, '  ')  # Adjust the indentation as needed

# Print complete LaTeX code
print(latex_complete)



\begin{table}[htbp]
\centering
\caption{Your table caption goes here}
\label{tab:your_table_label}
\begin{tabular}{llllll}
\toprule
  \begin{tabular}{lllrrl}
  \toprule
  Dataset & Metric & Test & Statistic & p-value & Significant \\
  \midrule
  ADNI & Weighted Avg. Shortest Path Length & ANOVA & 598.080717 & 0.000000 & Yes \\
  A4 & Weighted Avg. Shortest Path Length & ANOVA & 2085.018052 & 0.000000 & Yes \\
  ADNI & Weighted Clustering Coefficient & ANOVA & 567.441841 & 0.000000 & Yes \\
  A4 & Weighted Clustering Coefficient & ANOVA & 570.450427 & 0.000000 & Yes \\
  ADNI & Weighted Small World & Kruskal-Wallis & 1528.089792 & 0.000000 & Yes \\
  A4 & Weighted Small World & Kruskal-Wallis & 1954.486242 & 0.000000 & Yes \\
  \bottomrule
  \end{tabular}

\bottomrule
\end{tabular}
\end{table}



In [30]:
#separate for andi and a4

# Filter DataFrame for ADNI and A4 datasets
df_adni = results[results['Dataset'] == 'ADNI']
df_a4 =results[results['Dataset'] == 'A4']

# Convert DataFrames to LaTeX tables
latex_table_adni = df_adni.to_latex(index=False, escape=False)
latex_table_a4 = df_a4.to_latex(index=False, escape=False)

# Print LaTeX code for ADNI table
print("ADNI Table:")
print(latex_table_adni)

# Print LaTeX code for A4 table
print("\nA4 Table:")
print(latex_table_a4)

ADNI Table:
\begin{tabular}{lllrrl}
\toprule
Dataset & Metric & Test & Statistic & p-value & Significant \\
\midrule
ADNI & Weighted Avg. Shortest Path Length & ANOVA & 598.080717 & 0.000000 & Yes \\
ADNI & Weighted Clustering Coefficient & ANOVA & 567.441841 & 0.000000 & Yes \\
ADNI & Weighted Small World & Kruskal-Wallis & 1528.089792 & 0.000000 & Yes \\
\bottomrule
\end{tabular}


A4 Table:
\begin{tabular}{lllrrl}
\toprule
Dataset & Metric & Test & Statistic & p-value & Significant \\
\midrule
A4 & Weighted Avg. Shortest Path Length & ANOVA & 2085.018052 & 0.000000 & Yes \\
A4 & Weighted Clustering Coefficient & ANOVA & 570.450427 & 0.000000 & Yes \\
A4 & Weighted Small World & Kruskal-Wallis & 1954.486242 & 0.000000 & Yes \\
\bottomrule
\end{tabular}

